In [35]:
from networkit import *
import os
import numpy as np
import sys

## Read graph and coordinates

In [36]:
graphPath = "/home/harry/geographer-dev/meshes/slowrot-00000.graph"
G = readGraph(graphPath, Format.METIS)
print( G.numberOfNodes(), G.numberOfEdges() )

13112 19507


In [37]:
coordsFile = graphPath+".xyz"
coordsAll = [line.rstrip('\n') for line in open(coordsFile) if not line.startswith('%')]
print( coordsAll[:10] )

['416.000000 1013.333313 0.000000', '314.666656 437.333344 0.000000', '412.000000 382.666656 0.000000', '10.666667 352.000000 0.000000', '400.000000 133.333328 0.000000', '509.333344 328.000000 0.000000', '444.000000 569.333313 0.000000', '444.000000 574.666687 0.000000', '160.000000 10.666667 0.000000', '402.666656 261.333344 0.000000']


In [38]:
coordsX = [ float(x.split(' ')[0]) for x in coordsAll ]
print( max(coordsX) )

1018.666687


In [39]:
coordsY = [ float(x.split(' ')[1]) for x in coordsAll ]
print( max(coordsY) )

1013.333313


## Export data to gephi

In [40]:
client = gephi.streaming.GephiStreamingClient(url="http://localhost:8080/workspace1")

In [42]:
client.exportGraph(G)

In [43]:
client.exportNodeValues( G, coordsX, "X")
client.exportNodeValues( G, coordsY, "Y")

## Read partitions

In [45]:
direc = "/home/harry/supermuc/geographer-dev/experiments/output/"
filename = "slowrot-0000-random-5"

partitions = dict()

for file in os.listdir(direc):
    if file.startswith(filename) and file.endswith("part"):
        x = file.split(".")
        x = x[0].split("_")
        tool = x[2]
        print(tool)
        
        partition = [line.rstrip('\n') for line in open(direc+file)]
        #first line is comments, remove it. 
        #Alternatively, we can use: ... for line in open(...) if line not startswith("%")
        partition = [ int(x) for x in partition[1:] ]
        
        partitions[tool] = partition
        

Geographer
parMetisGeom
parMetisGraph
zoltanMJ
zoltanRCB
zoltanRIB


In [46]:
for x in partitions:
    client.exportNodeValues(G, partitions[x], x)

In [17]:
print( len(partitions["zoltanMJ"]))

13113


In [54]:
# read the partitions for the different hierarchy levels

levels = 3
allHier = list()
for i in range(1,levels):
    partFile = "/home/harry/geographer-dev/partResults/partHKM10_h" + str(i) + ".out"
    allHier.append( [line.rstrip('\n') for line in open(partFile)] )
    allHier[-1] = [ int(x) for x in allHier[-1] ] 
print( allHier[0][0:10] )    
print( allHier[1][0:10] )    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
